In [61]:
import os
import random
import numpy as np
print (np.__version__)
import pandas as pd
import math

from glob import glob
from IPython.display import display
from tqdm import tnrange, tqdm_notebook
import time
import datetime


1.15.4


#### Read dataset

In [2]:
#Defining original dataset directory.
Path = 'c://Datasets//Multiagent//rtb'

#Importing data.
os.chdir(Path)
Datasets_features = {}
for Filename in glob('*.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Train_ftr = Datasets_features['Train']
Validation_ftr = Datasets_features['Validation']

#### Define general functions

In [3]:

#Linear bidding strategy.
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))

def ortb_bidding(pCTR, c=50, lmda=5.2e-7, ortbtype=2):
    if ortbtype==2: 
        expr = (pCTR + np.sqrt(c ** 2 * lmda ** 2 + pCTR **2)) / (c * lmda)
        return c * (expr ** (1 / 3) - (1 / expr) ** (1 / 3))  
    else:
        return np.sqrt(c  / lmda * pCTR + c ** 2) - c

#Winning criterion #1.
def win_auction_1(bid, payprice):
    return bid >= payprice
    
#Winning criterion #2.
def win_auction_2(bid, payprice, Others):
    return ((bid >= payprice) and (bid >= max(Others)))
    
#Bidding strategy parameter eveluation.
def Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                         Pred_CTRs, prm, Dict_alg, key,alg):
    
    empCPM = []
    Bids, Cost, Imps, Clks = 0, 0, 0, 0
    
    #Looping on bid requests.
    for item in range(0, len(Clicks_Prices)):
        bid = 0
        Bids += 1
        pCTR = Pred_CTRs[item]           

            
        #Linear strategy.
        if key == 'Linear':
            Criterion = 1
            bid = linear_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        # NLogN strategy 
        elif key == 'NLogN':
            Criterion = 1
            bid = nlogn_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        elif key == 'Exp':
            Criterion = 1
            bid = exp_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        elif key == 'Ortb':
            Criterion = 1
            bid = ortb_bidding(pCTR, prm[0], prm[1],prm[2])
            Win = win_auction_1(bid, Clicks_Prices[item][1])                    
        #Exception: error.
        else:
            print('[Strategy error]: Wrong bidding strategy name!')
            break
            
        #Computing bid outcome.
        if Win:
            Imps += 1
            Clks += Clicks_Prices[item][0]
            
            if Criterion == 1:
                Cost += Clicks_Prices[item][1]/1000
                empCPM.append(Clicks_Prices[item][1])
            else:
                Cost += max(Clicks_Prices[item][1],max(others))/1000
                empCPM.append(max(Clicks_Prices[item][1],max(others)))
                
        #Stopping condition.
        if Cost > Budget:
            break
            
    #Reporting results.
    if Imps != 0: empCTR = Clks/Imps
    else: empCTR = 0
    
    avgCPM = np.mean(empCPM)
    if Clks != 0: avgCPC = np.sum(empCPM)/(Clks*1000)
    else: avgCPC = 0
    
    Output =  key + ',' + alg + ',' + str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
              + str(Bids) + ',' + str(Imps) + ',' + str(Clks) + ','\
              + str(empCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
    #temp_df = pd.DataFrame(,columns=['Algorithm', 'Parameter', 'Budget','Spent','Imps', 'Click','eCTR','avgCPM','avgCPC'])
    #curr_strat_vals.loc[len(curr_strat_vals)]=[]
    return Output, (key,alg,prm,Budget,Cost,Bids,Imps,Clks,empCTR,avgCPM,avgCPC)
    
#Bidding strategy simulation.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, Pred_CTRs,
                     file, Dict_alg, key, alg):
    parameters = Dict_alg[key]
    strategy_results = {}
    for idx in tnrange(len(parameters), desc='Params'):
        prm = parameters[idx]
        #print (idx,'/',len(parameters))     
        Output, result = Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                                      Pred_CTRs, prm, Dict_alg, key, alg)
        strategy_results[idx]=(result)
        file.write(Output + '\n')
    return strategy_results

#Actual bid creation and writing into file, based on baseB
def createBids(bidIds,prm,pred_CTRs,init_CTR,alg_name, key ):
    Filename = 'Team_07_'+ alg_name +'_'+ key+ '_bids' +'.csv'
    print('\tUsing best param...' + str(prm))
    with open(Filename, 'w') as file:
        file.write('bidid,bidprice' + '\n')
        for idx in range(len(bidIds)):
            bidId = bidIds[idx]
            pCTR  = pred_CTRs[idx]
            bid = 0
            if key == "Linear":
                bid   = linear_bidding(prm,pCTR, init_CTR)
            elif key == 'NLogN':
                bid = nlogn_bidding(prm, pCTR, init_CTR)
            elif key == 'Exp':
                bid = exp_bidding(prm,pCTR,init_CTR)
            elif key == 'Ortb':
                bid = ortb_bidding(pCTR, prm[0], prm[1],prm[2])
            else:
                raise NameError('key not found')
            
            #print(bidId,' ', baseBid,' ',pCTR,' ',init_CTR,' ', bid)
            file.write(bidId + ',' + '{:.1f}'.format(bid) +'\n')
            
    print('\t[File %s]: Process completed!' %Filename,'\n')

#### Read CTR information, base bid optimisation and bid creation
For CTR, two files should be in the folder, with the next format: '<Algorithm\>\_test\_pCTR.csv' and '<Algorithm\>\_validation\_pCTR.csv' (Note: Currently case sensitive). Then, the algorithm name shall be included in the _algs_ array.

In [72]:
import itertools

#Defining CTR directory.
Path = 'C://Datasets//Multiagent//calc//pctrs'

#Importing data.
os.chdir(Path)
Datasets_CTR = {}
overall_alg_vals = pd.DataFrame(columns=['BidStrat','Algorithm', 'Parameter', 'Budget','Spent','Bids','Imps', 'Click','eCTR','avgCPM','avgCPC'])

for Filename in glob('*_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Define algorithms to use
algs = ['ensemble_wavg'] #,'Catboost','NN', 'ensemble_avg','ensemble_wavg','LGBM''LGBM','LR'
#',


#Initialising values.
Init_CTR = sum(Train_ftr['click'])/len(Train_ftr)
Init_eCPC = sum(Train_ftr['payprice'])/(sum(Train_ftr['click'])*1000)
print("avgCTR: ",Init_CTR)

PathBB =   'c://Datasets//Multiagent//calc//bb'
PathBids = 'c://Datasets//Multiagent//calc//bids'
    
for alg in algs:
    #Defining variables.
    Test_CTR = Datasets_CTR[alg+'_test_pCTR']
    Validation_CTR = Datasets_CTR[alg+'_validation_pCTR']
    #Training bidding strategies.
    Budget = 6250
    random.seed(10)    
    
    #Defining general variables.
    Pred_CTRs = []
    Clicks_Prices = []

    #Initialising parameters.
    Prm_lin = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                              np.arange(100, 400, 10), np.arange(400, 800, 50)),
                              axis=0)
    
    Prm_log = np.logspace(1,4,num=25)
    #Prm_95 = [95]
    
    c_range = np.linspace(100, 240, 10) 
    lmda_range =  np.linspace(2e-6, 8e-6, 30)
    #np.concatenate((np.linspace(1e-7, 1e-5, 15),np.linspace(1e-5, 1e-3, 15),np.linspace(1e-3, 1e-1, 15)))
    range_100 = np.arange(100,120,0.5)
    #np.linspace(1e-8, 1e-4, 20)
    #c_range = np.linspace(8 5, 200, 15) 
    #lmda_range =   np.linspace(4e-6, 5e-6, 20)
    #{5.272631578947369e-06, 4.342857142857143e-06,3.342857142857143e-06}
    
    Prm_ortb = list(itertools.product(c_range,lmda_range,{2}))  
    
    Dict_alg = {'Ortb': Prm_ortb} #,,'Linear':range_100,  ,'Exp': Prm_lin,'NLogN': Prm_log,'Ortb': Prm_ortb,'Linear': range_100,'Ortb': Prm_ortb,
    
    Pred_CTRs = list(Validation_CTR['pCTR'].values)
    Pred_CTRs_Test = list(Test_CTR['pCTR'].values)
    Clicks_Prices = list(zip(Validation_ftr['click'], Validation_ftr['payprice']))

    #Training and saving results.
    os.chdir(PathBB)

    Header = 'bidStrat,algorithm,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC'
    print('pCTR estimator:', alg)
    for key in Dict_alg.keys():
        now = datetime.datetime.now()
        print('\t',key, 'bid  (Started', time.strftime("%H:%M:%S"),')' )
        os.chdir(PathBB)
        Filename = key + '_BB_'+ alg+'.csv'
        strategy_results = {}
        with open(Filename, 'w') as file:
            file.write(Header + '\n')
            strategy_results = Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                             Pred_CTRs, file, Dict_alg, key, alg)
        print('\t[File %s]: Process completed!' %Filename)
        strategy_results_df = pd.DataFrame.from_dict(strategy_results).transpose()
        strategy_results_df.rename(columns={0:'BidStrat', 1:'Algorithm', 2:'Parameter', 3:'Budget',4:'Spent',5:'Bids',
                                            6:'Imps', 7:'Click',8:'eCTR',9:'avgCPM',10:'avgCPC',11:'Agent'}, inplace=True)
        overall_alg_vals = overall_alg_vals.append(strategy_results_df.sort_values(['Click','eCTR'],ascending=False).head(3), sort=False)
        bestParam = strategy_results_df.sort_values(['Click','eCTR'],ascending=False).head(1).iloc[0,2]
        os.chdir(PathBids)
        createBids(Test_CTR['bidid'],bestParam,Pred_CTRs_Test,Init_CTR,alg,key)

print('\n• Full process completed!')
#print(overall_alg_vals.sort_values('Click', ascending=False))

avgCTR:  0.0007375623256619447
pCTR estimator: ensemble_wavg
	 Ortb bid  (Started 06:56:28 )



	[File Ortb_BB_ensemble_wavg.csv]: Process completed!
	Using best param...(240.0, 4.275862068965518e-06, 2)
	[File Team_07_ensemble_wavg_Ortb_bids.csv]: Process completed! 


• Full process completed!


In [73]:
overall_alg_vals.sort_values(['Click','eCTR'],ascending= False)

,BidStrat,Algorithm,Parameter,Budget,Spent,Bids,Imps,Click,eCTR,avgCPM,avgCPC
281,Ortb,ensemble_wavg,"(240.0, 4.275862068965518e-06, 2)",6250,6034.2,303925,128792,165,0.00128114,46.8523,36.5709
220,Ortb,ensemble_wavg,"(208.88888888888889, 4.0689655172413795e-06, 2)",6250,6250.04,303847,132845,165,0.00124205,47.0476,37.879
252,Ortb,ensemble_wavg,"(224.44444444444446, 4.482758620689655e-06, 2)",6250,5665.83,303925,123529,164,0.00132762,45.8664,34.5477


In [19]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [16]:
range100 = np.arange(100,130,0.5)
range100

array([100. , 100.5, 101. , 101.5, 102. , 102.5, 103. , 103.5, 104. ,
       104.5, 105. , 105.5, 106. , 106.5, 107. , 107.5, 108. , 108.5,
       109. , 109.5, 110. , 110.5, 111. , 111.5, 112. , 112.5, 113. ,
       113.5, 114. , 114.5, 115. , 115.5, 116. , 116.5, 117. , 117.5,
       118. , 118.5, 119. , 119.5, 120. , 120.5, 121. , 121.5, 122. ,
       122.5, 123. , 123.5, 124. , 124.5, 125. , 125.5, 126. , 126.5,
       127. , 127.5, 128. , 128.5, 129. , 129.5])

In [33]:
strategy_results_df.sort_values(['Click','Spent'],ascending= [False,True])

,BidStrat,Algorithm,Parameter,Budget,Spent,Bids,Imps,Click,eCTR,avgCPM,avgCPC
11,Ortb,LGBM,"(180.0, 3.757894736842105e-06, 2)",6250,6125.55,303925,142473,162,0.00113706,42.9945,37.8121
53,Ortb,LGBM,"(195.55555555555554, 3.789473684210526e-06, 2)",6250,6153.22,303925,142456,162,0.00113719,43.1938,37.9828
221,Ortb,LGBM,"(257.77777777777777, 3.9157894736842104e-06, 2)",6250,6158.28,303925,141393,162,0.00114574,43.5543,38.0141
179,Ortb,LGBM,"(242.22222222222223, 3.884210526315789e-06, 2)",6250,6169.05,303925,141764,162,0.00114274,43.5163,38.0805
95,Ortb,LGBM,"(211.11111111111111, 3.821052631578947e-06, 2)",6250,6169.97,303925,142339,162,0.00113813,43.347,38.0862
137,Ortb,LGBM,"(226.66666666666666, 3.852631578947368e-06, 2)",6250,6172.68,303925,142079,162,0.00114021,43.4454,38.103
9,Ortb,LGBM,"(180.0, 3.726315789473684e-06, 2)",6250,6180.95,303925,143280,162,0.00113065,43.1389,38.154
261,Ortb,LGBM,"(273.3333333333333, 3.9157894736842104e-06, 2)",6250,6192.05,303925,141704,162,0.00114323,43.6971,38.2225
343,Ortb,LGBM,"(304.44444444444446, 3.9473684210526315e-06, 2)",6250,6192.2,303925,141419,162,0.00114553,43.7862,38.2235
219,Ortb,LGBM,"(257.77777777777777, 3.884210526315789e-06, 2)",6250,6211.29,303925,142146,162,0.00113967,43.6965,38.3413
